In [1]:
# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 필요한 패키지와 라이브러리를 가져옴
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 60.6MB/s 
     |████████████████████████████████| 92kB 10.5MB/s 
     |████████████████████████████████| 460kB 34.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
# 구글드라이브 폴더 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#파일이 위치한 경로로 이동
%cd /content/drive/MyDrive/06.대학원/08.최신정보검색기술/정보검색기술_프로젝트/Model_develop/신한_

/content/drive/MyDrive/06.대학원/08.최신정보검색기술/정보검색기술_프로젝트/Model_develop/신한_


In [5]:
import pandas as pd
import numpy as np

import os
import sys

sys.path.append("..")

# 토큰화 셋팅
#from util.text_preprocessing import load_data, tokenize_okt, count_comment
from util.text_preprocessing import tokenize_okt, count_comment

#토큰화
import tqdm

In [6]:
!ls

신한지주	      naver_w2v		      sh_w2v_model	 util
gisa3.csv	      naver_w2v_metadata.tsv  sh_word2vec.model  word2vec.html
gisa3.txt	      naver_w2v_tensor.tsv    test2.ipynb	 Word2Vec.ipynb
korean_stopwords.txt  sh_ft		      text.csv


In [7]:
#csv 파일 Read
sh_df = pd.read_csv('./gisa3.csv', error_bad_lines=False)

In [8]:
sh_df.sample(5)

,Unnamed: 0,Unnamed: 0.1,title,url,label
628,628,628,Z세대 마음잡기 나선 은행권...디지털 플랫폼 특화 공략 본격화,http://www.updownnews.co.kr/news/articleView.h...,0
391,391,391,`전자금융거래 검증 미흡` 신한銀·한투증권 과태료,http://www.dt.co.kr/contents.html?article_no=2...,0
277,277,277,"신한은행, 빅데이터 활용해 상권분석 컨설팅",http://www.fnnews.com/news/202105151008332569,0
315,315,315,커스터디 사업 놓고 속앓이하는 은행,http://www.etnews.com/20210518000145,0
465,465,465,"H2KOREA, 수소산업 확대 위해 신한은행과 맞손",https://view.asiae.co.kr/article/2021051016344...,0


Tokenize comments

In [9]:
token_data = []

for comment in tqdm.tqdm(sh_df.title):
    #print(comment)
    token = tokenize_okt(comment)
    token_data.append(token)

100%|██████████| 1000/1000 [00:12<00:00, 77.02it/s]


In [10]:
keys, n_vocab = count_comment(token_data)


신한은행(347) 금융(92) 은행(91) 신한(73) 서비스(52) 대출(52) 출시(49) 은행권(46) 네이버(44) 업무(44) 선물(42) 전세(39) 스타트업(35) 지갑(34) 디지털(33) 해외(32) 지원(32) 모집(31) 온(30) 컨설팅(29) 분석(29) 환(29) 특성화고(29) 커리어(29) 페이(28) 백신(28) 플랫폼(28) 위(27) 체결(27) 자산(26) 접종(26) 상권(26) 활용(26) 증명(25) 주식(25) 협약(25) 통계청(25) 데이터(25) 거래(25) 기(25) 비대(25) 자영(24) 업자(24) 확인(24) 인재(24) 국민은행(23) 채용(23) 빅데이터(22) 코로나(21) 기업(21) 
Total Vocab:  998



In [11]:
series_token_data = pd.Series(token_data)
series_token_data[:5]

0                      [국민은행, 연속, 경영, 평가, 최우수, 기업, 선정]
1    [방금, 앱, 상담, 만족하다, 신한, 은행권, 첫, 디지털, 고객, 경험, 만족도...
2                     [금융, 스타트업, 비즈니스, 네트워킹, 프로그램, 개최]
3          [문과, 생, 컴, 공생, 모시다, 공채, 대신, 인력, 수시, 채용, 은행]
4         [배달, 통, 가다, 신한은행, 티몬, 오다, 배달, 앱, 시장, 춘추전국시대]
dtype: object

In [12]:
sh_df = sh_df.reset_index(drop=True)
sh_df['comment_token'] = series_token_data
sh_df.head()

,Unnamed: 0,Unnamed: 0.1,title,url,label,comment_token
0,0,0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m,0,"[국민은행, 연속, 경영, 평가, 최우수, 기업, 선정]"
1,1,1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...,0,"[방금, 앱, 상담, 만족하다, 신한, 은행권, 첫, 디지털, 고객, 경험, 만족도..."
2,2,2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...,0,"[금융, 스타트업, 비즈니스, 네트워킹, 프로그램, 개최]"
3,3,3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...,0,"[문과, 생, 컴, 공생, 모시다, 공채, 대신, 인력, 수시, 채용, 은행]"
4,4,4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...,0,"[배달, 통, 가다, 신한은행, 티몬, 오다, 배달, 앱, 시장, 춘추전국시대]"


In [13]:
tokens = sh_df['comment_token']
tokens[:5]

0                      [국민은행, 연속, 경영, 평가, 최우수, 기업, 선정]
1    [방금, 앱, 상담, 만족하다, 신한, 은행권, 첫, 디지털, 고객, 경험, 만족도...
2                     [금융, 스타트업, 비즈니스, 네트워킹, 프로그램, 개최]
3          [문과, 생, 컴, 공생, 모시다, 공채, 대신, 인력, 수시, 채용, 은행]
4         [배달, 통, 가다, 신한은행, 티몬, 오다, 배달, 앱, 시장, 춘추전국시대]
Name: comment_token, dtype: object

In [14]:
import codecs
from gensim.models import word2vec
from gensim.models import fasttext

In [15]:
import multiprocessing
config = {
    'min_count': 1,  # 등장 횟수가 1 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 20,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
    'window': 5,
    'seed': 25 #random number,
}

In [16]:
# 모델 학습
model_w2v = word2vec.Word2Vec(tokens,**config)

In [17]:
#모델 저장
model_w2v.save(os.path.join("sh_w2v_model"))

In [18]:
# 모델 불러오기
w2v_model = word2vec.Word2Vec.load("sh_w2v_model")

In [19]:

w2v_model.wv.most_similar(positive = "신한")

[('찾다', 0.874783992767334),
 ('커리어', 0.8698651194572449),
 ('온', 0.8628479838371277),
 ('모집', 0.8617465496063232),
 ('기', 0.8521402478218079),
 ('특성화고', 0.8472799062728882),
 ('한번', 0.8459292650222778),
 ('학생', 0.8390078544616699),
 ('절세', 0.8372265100479126),
 ('커리', 0.8361961841583252)]


fastText 모델 학습 및 불러오기
학습 데이터와 파라미터 동일하게 해서 이후에 word2vec과 결과 비교
자모 단위 모델 추가

In [20]:
print(tokens[0])
print(len(tokens))

['국민은행', '연속', '경영', '평가', '최우수', '기업', '선정']
1000


In [21]:
ft_config = {
    'min_count': 1,  # 등장 횟수가 1 이하인 단어는 무시
    'size': 300,  # 300차원짜리 벡터스페이스에 embedding
    'sg': 1,  # 0이면 CBOW, 1이면 skip-gram을 사용한다
    'batch_words': 10000,  # 사전을 구축할때 한번에 읽을 단어 수
    'iter': 100,  # 보통 딥러닝에서 말하는 epoch과 비슷한, 반복 횟수
    'workers': multiprocessing.cpu_count(),
    'window': 5,
    'seed': 25, #random number,
    'word_ngrams':1 # uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec.
}

In [22]:

# 모델 학습
fastText_model = fasttext.FastText(sentences=tokens, **ft_config)

In [23]:
# 모델 저장
fastText_model.save('sh_ft')

In [24]:
# 모델 불러오기
fastText_model = fasttext.FastText.load('sh_ft')

In [25]:
# 연관성 테스트
fastText_model.wv.most_similar("신한")

[('동행', 0.5132623314857483),
 ('혜택', 0.49092695116996765),
 ('절세', 0.4906260073184967),
 ('번', 0.4901426434516907),
 ('정답', 0.48787716031074524),
 ('에스', 0.4867488741874695),
 ('인턴십', 0.48079437017440796),
 ('연결', 0.47865059971809387),
 ('궁합', 0.475081205368042),
 ('겜', 0.4741555452346802)]

Word 시각화 수행

In [26]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)#벡터를 2차원으로 축소

In [39]:
import matplotlib.font_manager as fm
fm._rebuild()

plt.rc('font', family='NanumGothic')

In [47]:
w2v_vectors = w2v_model.wv
ft_vectors = fastText_model.wv

In [48]:
w_vocabs = w2v_vectors.vocab.keys()
w_vectors_list = [w2v_vectors[v] for v in w_vocabs]

f_vocabs = ft_vectors.vocab.keys()
f_vectors_list = [ft_vectors[v] for v in f_vocabs]

In [34]:
xys = pca.fit_transform(w_vectors_list)
xs = xys[:,0]
ys = xys[:,1]

In [43]:
# plotly 이용해서 html 파일로 만들기 
# annotation text 만들기 (시각화할 때 벡터 말고 단어도 필요하니까)
# vocabs = word_vectors.vocab.keys()

text=[]
for i,v in enumerate(w_vocabs):
    text.append(v)

In [44]:
import plotly
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=xs,
                                y=ys,
                                mode='markers+text',
                                text=text)) 

fig.update_layout(title='Word2Vec')
fig.show()

plotly.offline.plot(
fig, filename='word2vec.html'
)

'word2vec.html'

In [52]:
xys = pca.fit_transform(f_vectors_list)
xs = xys[:,0]
ys = xys[:,1]

In [53]:
import plotly
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=xs,
                                y=ys,
                                mode='markers+text',
                                text=text)) 

fig.update_layout(title='FastText')
fig.show()

plotly.offline.plot(
fig, filename='FastText.html'
)

'FastText.html'

지수인덱스 만들기

In [54]:
# 라벨 0/1/-1
sh_df 

,Unnamed: 0,Unnamed: 0.1,title,url,label,comment_token
0,0,0,KB국민은행 2년 연속 경영평가 최우수기업 선정,http://yna.kr/AKR20210522035700003?did=1195m,0,"[국민은행, 연속, 경영, 평가, 최우수, 기업, 선정]"
1,1,1,"""방금 앱상담, 만족하셨나요?"" 신한銀, 은행권 첫 '디지털 고객경험 만족도 조사'",https://www.hankyung.com/economy/article/20210...,0,"[방금, 앱, 상담, 만족하다, 신한, 은행권, 첫, 디지털, 고객, 경험, 만족도..."
2,2,2,"신한금융, 스타트업 비즈니스 네트워킹 프로그램 개최",http://www.edaily.co.kr/news/newspath.asp?news...,0,"[금융, 스타트업, 비즈니스, 네트워킹, 프로그램, 개최]"
3,3,3,“문과생 말고 컴공생 모셔요”… 공채 대신 IT 인력 수시 채용하는 은행들,https://biz.chosun.com/stock/finance/2021/05/2...,0,"[문과, 생, 컴, 공생, 모시다, 공채, 대신, 인력, 수시, 채용, 은행]"
4,4,4,"'배달통' 가고, 신한은행·티몬 오고… 배달앱 시장 춘추전국시대",http://www.viva100.com/main/view.php?key=20210...,0,"[배달, 통, 가다, 신한은행, 티몬, 오다, 배달, 앱, 시장, 춘추전국시대]"
...,...,...,...,...,...,...
995,995,995,"신한은행, 통계청과 빅데이터 기반 사회 문제 공동연구 진행",http://www.nspna.com/news/?mode=view&newsid=50...,-1,"[신한은행, 통계청, 빅데이터, 기반, 사회, 문제, 공동, 연구, 진행]"
996,996,996,"신한금융그룹, 정책형 뉴딜펀드 '첫' 투자 실행 완료",http://news.tf.co.kr/read/economy/1856746.htm,0,"[금융, 그룹, 정책, 뉴딜, 펀드, 첫, 투자, 실행, 완료]"
997,997,997,"신한은행, '신한 S Life Care' 증여신탁 출시",http://www.thebigdata.co.kr/view.php?ud=202105...,1,"[신한은행, 신한, 증여, 신탁, 출시]"
998,998,998,"신한은행, 학대피해아동쉼터에 차량 전달",http://www.enewstoday.co.kr/news/articleView.h...,-1,"[신한은행, 학대, 피해, 아동, 쉬다, 터, 차량, 전달]"


In [59]:
# 라벨 개수 세기
sh_df["label"].value_counts()

 0    704
 1    265
-1     31
Name: label, dtype: int64

In [61]:
len(sh_df)

1000

In [64]:
for i in range(10):
  print(i)
  start = i +1
  end = (i+1) * 100
  data_

0
1
2
3
4
5
6
7
8
9
